# Introduction to Decision Trees

Decision trees are a powerful and popular machine learning technique. The basic concept is very similar to trees that we commonly use to aid decision-making.

Machine Learning techniques enables us to automatically construct a decision tree that tells us what outcomes we should predict in certain situations.

The decision tree algorithm is a supervised learning algorithm -- we first construct the tree with historical data, and then use it to predict an outcome. 
- One of the major advantages of decision trees is that they can pick up nonlinear interactions between variables in the data that linear regression can't.

The following work is a walk through of the building blocks of making a decision tree automatically.

We'll be looking at individual income in the United States. 
The data is from the 1994 census, and contains information on an individual's 
- marital status
- age 
- type of work, and more. 

The target column, or what we want to predict, is whether individuals make less than or equal to 50k a year, or more than 50k a year. The dataset can be accessed from the [University of California, Irvine's website](http://archive.ics.uci.edu/ml/datasets/Adult).

In [95]:
import pandas as pd

# First column is age of the person. Set index_col to False to avoid pandas thinking that the first column is row indexes
income = pd.read_csv("income.csv", index_col=False)
print(income.head(5))

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

We have categorical variables such as workclass that have string values. 
- Multiple individuals can share the same string value. The types of work include State-gov, Self-emp-not-inc, Private, and so on. Each of these strings is a label for a category. 
- Another example of a column of categories is sex, where the options are Male and Female.

Before we get started with decision trees, we need to convert the categorical variables in our data set to numeric variables.

In [96]:
workclass_col = pd.Categorical(income['workclass'])
income['workclass'] = workclass_col.codes
income['workclass'].head()

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8

Similarly, we will convert all the other categorical variables to numeric variables.

In [97]:
def convert_categorical(column,data):
#     for each in columns:
    column_cat = pd.Categorical(data[column])
    codes = column_cat.codes
    return codes
    
income['education'] = convert_categorical('education',income)
income['marital_status'] = convert_categorical('marital_status',income)
income['occupation'] = convert_categorical('occupation',income)
income['relationship'] = convert_categorical('relationship',income)
income['race'] = convert_categorical('race',income)
income['sex'] = convert_categorical('sex',income)
income['native_country'] = convert_categorical('native_country',income)
income['high_income'] = convert_categorical('high_income',income)


In [98]:
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


Now think of rows of data flowing through a decision tree. We can split the data set into two portions based on whether the individual works in the private sector or not.

In [99]:
# Split income into two parts based on the value of the workclass column
private_incomes = income[income['workclass']==4]
public_incomes = income[~(income['workclass']==4)]
print(private_incomes.shape[0])
print(public_incomes.shape[0])

22696
9865


A decision tree is made up of a series of nodes and branches. A node is where we split the data based on a variable, and a branch is one side of the split. 

The nodes at the bottom of the tree, where we decide to stop splitting, are called terminal nodes, or leaves.
- When we are splitting the data, we are not doing it randomly. We have a motive in mind- an objective. Out goal is to make predictions on future data. 
    - In order to do this, all rows in each leaf must have only one value for our target column.
    
Each leaf can only have rows with the same values for our target column. If this isn't the case, we won't be able to make effective predictions.

After constructing a tree using the data, we'll want to make predictions. In order to do this, we'll take a new row and feed it through our decision tree.

### Entropy
Now that we have a high-level view of how decision trees work, let's explore the details and learn how to perform the splits.

Because we're trying to reach the leaves having only 1s or only 0s in high_income, each split will need to get us closer to that goal.

When we split, we'll try to separate as many 0s from 1s in the high_income column as we can. In order to do this, we need a metric for how "together" the different values in the high_income column are.

We will use a metric called entropy for this purpose. 
- __Entropy refers to disorder.__ 
- The more "mixed together" 1s and 0s are, the higher the entropy. A data set consisting entirely of 1s in the high_income column would have low entropy.

Entropy comes from information theory. 
- Information theory is based on probability and statistics, and deals with the transmission, processing, utilization, and extraction of information. A key concept in information theory is the notion of a bit of information. 
- One bit of information is one unit of information.

We can represent a bit of information as a binary number because it either has the value 1 or 0. Suppose there's an equal probability of a coin toss being heads (1) or not heads (0). If I tell you that it will be heads, I've given you one bit of information.

The formula for entropy looks like this:
−∑P(xi)logP(xi)


In [100]:
# Computing the entropy of the high_income column in the income dataframe

import math

high_income_0 = len(income[income['high_income']==0])
high_income_1 = len(income[income['high_income']==1])
high_income_len = len(income['high_income'])
income_entropy= -((high_income_0/high_income_len) * math.log(high_income_0/high_income_len, 2) + high_income_1/high_income_len * math.log(high_income_1/high_income_len, 2))

print(income_entropy)

0.7963839552022132


We need a way to go from calculating the entropy to finding the variables to split on.  We can do this using information gain, which tells us which split will reduce entropy the most.

Here's the formula for information gain:

IG(T,A)=Entropy(T)−∑v∈A(|Tv|/|T|)*Entropy(Tv)

To compute it, we first calculate the entropy for T. 
- Then, for each unique value v in the variable A, we compute the number of rows in which A takes on the value v , and divide it by the total number of rows. 
- Next, we multiply the results by the entropy of the rows where A is v. 
- We add all of these subset entropies together, then subtract from the overall entropy to get information gain.

__We're finding the entropy of each set post-split, weighting it by the number of items in each split, then subtracting from the current entropy. If the result is positive, we've lowered entropy with our split. The higher the result is, the more we've lowered entropy.__

- One strategy for constructing trees is to create as many branches at each node as there are unique values for the variable we're splitting on. This approach usually involves more complexity than it's worth and doesn't improve prediction accuracy.

- To make calculations simpler, we won't do it for each unique value. We'll find the median for the variable we're splitting on instead. Any rows where the value of the variable is below the median will go to the left branch, and the rest of the rows will go to the right branch. To compute information gain, we'll only have to compute entropies for two subsets.



In [101]:
# Compute the information gain for splitting on the age column of income

import numpy

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = numpy.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

median_age = income['age'].median()

left_split = income[income['age']<=median_age]
right_split = income[income['age']>median_age]

left_splt_len =len(left_split['age'])
right_split_len = len(right_split['age'])
income_len = len(income['age'])


income_entropy = calc_entropy(income['high_income'])
age_information_gain = income_entropy -(((left_splt_len/income_len)*calc_entropy(left_split['high_income']))
                                        + ((right_split_len/income_len)*calc_entropy(right_split['high_income'])))

age_information_gain

0.047028661304691965

Now that we know how to compute information gain, we can determine the best variable to split a node on. When we start our tree, we want to make an initial split. We'll find the variable to split on by calculating which split would have the highest information gain.

Create a list called information_gains.
- It should contain, in order, the information gain from splitting on these columns: age, workclass, education_num, marital_status, occupation, relationship, race, sex, hours_per_week, native_country.

- Find the highest value in the information_gains list, and assign the corresponding column name to highest_gain.

In [102]:
def calc_information_gain(data,split_column,target_column):
    
    original_entropy = calc_entropy(data[target_column])
    
    column = data[split_column]
    median = column.median()
    
    left_split = data[column <= median]
    right_split = data[column > median]
    
    substract = 0
    for each in [left_split, right_split]:
        prob = each.shape[0]/data.shape[0]
        substract+= prob * calc_entropy(each[target_column])
    
    return original_entropy - substract

In [103]:
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

information_gains = [calc_information_gain(income, each, "high_income") for each in columns]

highest_gain = columns[information_gains.index(max(information_gains))]
highest_gain

'marital_status'

So, 'marital_status' gives us the best split of data to form a tree to find wheather a person has a high income. 

Now we know how to make a single split. To construct an entire tree, we'll need to continue creating splits until the leaves only have a single class.

So far, we've been following the ID3 algorithm to construct decision trees. There are other algorithms like CART that use different measures for the split criterion.
Now, we'll figure out how to recursively construct an entire tree using the ID3 algorithm.

- We will use the concept of recursion to create a ID3 algorithm. 
- In general, recursion is the process of splitting a large problem into smaller chunks. Recursive functions will call themselves, then combine the results into a final output.
- Building a tree is a perfect use case for recursive algorithms. At each node, we'll call a recursive function that will split the data into two branches. Each branch will lead to a node, and the function will call itself to build the tree out.

### Determining the Column to Split On
We already have functions to calculate entropy and information gain.

Now we need a function that returns the name of the column we should use to split a data set. The function should take the name of the data set, the target column, and a list of columns we might want to split on as input.

In [104]:
def find_best_column(data, target_column, columns):
    
    # Calcualate information gain to find the column that splits the data best
    information_gains = [calc_information_gain(data, each, target_column) for each in columns]
    
    highest_gain = columns[information_gains.index(max(information_gains))]
    return highest_gain
    
    
    
# A list of columns to potentially split income with
columns = ["age", "workclass", "education_num", 
           "marital_status", "occupation", "relationship", 
           "race", "sex", "hours_per_week", "native_country"]
target_column = 'high_income'

income_split = find_best_column(income, target_column, columns)
income_split

'marital_status'

Looks like marital_status is the best column to split our data on.


### Creating the Recursive Fucntion to Implement the id3 Algorithm

In [105]:
# Lists to store our labels for nodes
label_1s = []
label_0s = []

def id3(data, target_column, columns):
    unique_targets = pd.unique(data[target_column])
    
    if len(unique_targets) == 1:
        if unique_targets == 1:
            label_1s.append(1)
            return label_1s
        elif unique_targets == 0:
            label_0s.append(0)
            return label_0s
    
    best_column = find_best_column(data, target_column, columns)
    
    # Finding median value of the best column evaluated in the previous step.
    column_median = data[best_column].median()
    
    # Split the data based on the basis of the column_median
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    # Recursively applying the ID3 alogrithm
    for split in (left_split, right_split):
        id3(split, target_column, columns)
        

In [106]:
# Create the data set that we used in the example on the last screen
data = pd.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"])

In [107]:
print(label_1s)
print(label_0s) 

[1, 1, 1]
[0, 0, 0]


Now we can store the entire tree, rather than the leaf labels only.  
- We'll use nested dictionaries to store the entire tree. 
- We can represent the root node with a dictionary, and branches with the keys left and right. 
- We'll store the column we're splitting on as the key column, and the median value as the key median. 
- Finally, we can store the label for a leaf as the key label. We'll also number each node as we go along using the number key.

In [108]:
# Create a dictionary to hold the tree
tree = {}

# Create an empty list to store the node values
node = []

def id3(data, target_column, columns, tree):
    unique_targets = pd.unique(data[target_column])
    
    node.append(len(node)+1)
    tree['number'] = node[-1]
    
    if len(unique_targets) == 1:
        if unique_targets == 1:
            tree['label'] = 1
            return tree
        elif unique_targets == 0:
            tree['label'] = 0
            return tree
            
    best_column = find_best_column(data, target_column, columns)
    column_median = data[best_column].median()
    
    tree['column'] = best_column
    tree['median'] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_data = [['left', left_split],['right', right_split]]
    
    for name, split in split_data:
        # Creating an empty tree like we did initially. This empty tree will be passed recursively to the id3 function
        tree[name] = {}
        id3(split, target_column, columns, tree[name])

In [109]:
# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"], tree)

In [110]:
tree

{'column': 'age',
 'left': {'column': 'age',
  'left': {'column': 'age',
   'left': {'label': 0, 'number': 4},
   'median': 22.5,
   'number': 3,
   'right': {'label': 1, 'number': 5}},
  'median': 25.0,
  'number': 2,
  'right': {'label': 1, 'number': 6}},
 'median': 37.5,
 'number': 1,
 'right': {'column': 'age',
  'left': {'column': 'age',
   'left': {'label': 0, 'number': 9},
   'median': 47.5,
   'number': 8,
   'right': {'label': 1, 'number': 10}},
  'median': 55.0,
  'number': 7,
  'right': {'label': 0, 'number': 11}}}

The tree dictionary shows all of the relevant information, but it doesn't look very nice. We can fix its appearance by printing it out in a nicer format.

To do this, we'll need to recursively iterate through our tree dictionary. Any dictionary that has a label key is a leaf. Whenever we find one, we'll print out the label. Otherwise, we'll loop through the tree's left and right keys and recursively call the same function.

We also need to keep track of a depth variable. This variable will allow us to use indentation to indicate the order of the nodes. Before we print anything out, we'll prefix it with the number of spaces corresponding to the depth variable.

In [111]:
# Funtion to be called at a node where label is found
def print_node(string, depth):
    indent = "    "* depth
    print('{0}{1}'.format(indent, string))

# Funciton to print each node
def print_tree(tree, depth):
    
    # Check if the node has a label
    if 'label' in tree:
        print_node("Leaf: Label {0}".format(tree['label']),depth)
        return
    
    print_node("{0} > {1}".format(tree['column'],tree['median']),depth)
    
    tree_branches = [tree['left'],tree['right']]
    
    for branch in tree_branches:
        depth+=1
        print_tree(branch, depth)
        

In [112]:
print_tree(tree, 0)

age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
                Leaf: Label 1
            Leaf: Label 1
        age > 55.0
            age > 47.5
                Leaf: Label 0
                    Leaf: Label 1
                Leaf: Label 0


Each node prints the criteria on which it was split. 
So, how to predict a new value by looking at this tree?

Let's say we want to predict the following row-
age : 50 and marital_status : 1

- First, we'd split on age > 37.5 and go to the right. Then, we'd split on age > 55.0 and go to the left. Then, we'd split on age > 47.5 and go to the right. We'd end up predicting a 1 for high_income.

Making predictions with such a small tree is fairly straightforward, but with the increase in data size, it becomes difficult to eyeball predictions. We'd want an automated way to do this instead.

### Making Predictions Using the Tree

Following is a function that makes predictions automatically. All we need to do is follow the split points we've already defined with a new row.

In [124]:
def predictions(tree, row):
    
    if 'label' in tree:
        return tree['label']
    
    column = tree["column"]
    median = tree["median"]
    
    if row[column] <= median:
        return predictions(tree['left'],row)
        
    elif row[column] > median:
        return predictions(tree['right'],row)
     

In [125]:
print(predictions(tree, data.iloc[0]))

0


In [126]:
# Make predicitons for multiple rows 
new_data = pd.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data
new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    return df.apply(lambda row : predictions(tree,row) ,axis = 1)


predict = batch_predict(tree, new_data)

In [127]:
print(predict)

0    0
1    0
2    0
3    0
4    1
5    0
dtype: int64
